In [1]:
import numpy as np

In [2]:
#lendo arquivos
arquivo = open("instancia1.txt", 'r')
texto = arquivo.readlines()
arquivo.close()
print(texto)

['8\n', '400\n', '350\n', '120000\n', '450\n', '180\n', '50\n', '100\n', '200\n', '100\n', '440\n', '250\n', '220\n', '350\n', '180\n', '250']


In [3]:
#salvando lista de semanas em T
t = int(texto[0].split()[0])
T = []
for i in range(t):
    T.append(int(i+1))

print("Semanas:",T)

#Salvando preços de C1, C2
CC1 = int(texto[1].split()[0])
CC2 = int(texto[2].split()[0])
print("Custo de compra de C1:", CC1, "\nCusto de compra de C2:", CC2)

#Salvando custo fixo por pedido
CF = int(texto[3].split()[0])
print("Custo fixo por pedido:", CF)

#Salvando custos de estoques
VP1 = int(texto[4].split()[0])
VP2 = int(texto[5].split()[0])
VC1 = int(texto[6].split()[0])
VC2 = int(texto[7].split()[0])
print("VP1 =", VP1, "\nVP2 =", VP2, "\nVC1 =", VC1, "\nVC2 =", VC2)

#Salvando as demandas de P1
d = []

for i in range(8, len(texto)):
    d.append(int(texto[i].split()[0]))

for i in range(t):
    demanda = dict(zip(T, d))
    
print("Demandas:", demanda)

Semanas: [1, 2, 3, 4, 5, 6, 7, 8]
Custo de compra de C1: 400 
Custo de compra de C2: 350
Custo fixo por pedido: 120000
VP1 = 450 
VP2 = 180 
VC1 = 50 
VC2 = 100
Demandas: {1: 200, 2: 100, 3: 440, 4: 250, 5: 220, 6: 350, 7: 180, 8: 250}


In [4]:
#!pip install gurobipy
from gurobipy import *

#modelo
model = Model('MPS')

#criando as variaveis
EP1t = model.addVars(T, vtype=GRB.INTEGER, name = 'EP1t')
EP2t = model.addVars(T, vtype=GRB.INTEGER, name = 'EP2t')
EC1t = model.addVars(T, vtype=GRB.INTEGER, name = 'EC1t')
EC2t = model.addVars(T, vtype=GRB.INTEGER, name = 'EC2t')
P1t = model.addVars(T, vtype=GRB.INTEGER, name = 'P1t')
P2t = model.addVars(T, vtype=GRB.INTEGER, name = 'P2t')
C1t = model.addVars(T, vtype=GRB.INTEGER, name = 'C1t')
C2t = model.addVars(T, vtype=GRB.INTEGER, name = 'C2t')
CFt = model.addVars(T, vtype=GRB.BINARY, name = 'CFt')

Restricted license - for non-production use only - expires 2022-01-13


In [5]:
#definindo a função objetivo
som1c1 = quicksum(CC1*C1t[i] for i in T)
som1c2 = quicksum(CC2*C2t[i] for i in T)
som1 = som1c1 + som1c2

som2EP1t = quicksum(VP1*EP1t[i] for i in T)
som2EP2t = quicksum(VP2*EP2t[i] for i in T)
som2 = som2EP1t + som2EP2t

som3EC1t = quicksum(VC1*EC1t[i] for i in T)
som3EC2t = quicksum(VC2*EC2t[i] for i in T)
som3 = som3EC1t + som3EC2t

z = som1 + som2 + som3
model.setObjective(z, GRB.MINIMIZE)

In [6]:
#definindo as restrições
model.addConstrs(EP1t[i]==EP1t[i-1]+P1t[i]+demanda[i] for i in T if i!=1)
model.addConstrs(EP2t[i]==EP2t[i-1]+P2t[i]+(demanda[i]*2) for i in T if i!=1)
model.addConstrs(EC1t[i]==EC1t[i-1]+C1t[i]-P2t[i]+(demanda[i]*3) for i in T if i!=1)
model.addConstrs(EC2t[i]==EC2t[i-1]+C2t[i]+(P2t[i]*2) for i in T if i!=1)

model.addConstrs((C1t[i]+C2t[i])+CF <= CFt[i]*1000000 for i in T)
model.addConstrs(P1t[i]>=demanda[i]-EP1t[i] for i in T)
model.addConstrs(P2t[i]>=(demanda[i]*2)-EP2t[i] for i in T)
model.addConstrs(C1t[i]>=P2t[i]+(demanda[i]*2)-EC1t[i] for i in T)
model.addConstrs(C2t[i]>=(P2t[i]*2)-EC2t[i] for i in T)
model.addConstrs(P1t[i] <= 800 for i in T)


#resolver:
model.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 76 rows, 72 columns and 210 nonzeros
Model fingerprint: 0xa41ac297
Variable types: 0 continuous, 72 integer (8 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  Objective range  [5e+01, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+02, 1e+05]
Found heuristic solution: objective 7580800.0000
Presolve removed 76 rows and 72 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds
Thread count was 1 (of 4 available processors)

Solution count 2: 7.4448e+06 7.5808e+06 

Optimal solution found (tolerance 1.00e-04)
Best objective 7.444800000000e+06, best bound 7.444800000000e+06, gap 0.0000%


In [7]:
#display
model.display()

Minimize
   <gurobi.LinExpr: 450.0 EP1t[1] + 450.0 EP1t[2] + 450.0 EP1t[3] + 450.0 EP1t[4] + 450.0 EP1t[5] + 450.0 EP1t[6] + 450.0 EP1t[7] + 450.0 EP1t[8] + 180.0 EP2t[1] + 180.0 EP2t[2] + 180.0 EP2t[3] + 180.0 EP2t[4] + 180.0 EP2t[5] + 180.0 EP2t[6] + 180.0 EP2t[7] + 180.0 EP2t[8] + 50.0 EC1t[1] + 50.0 EC1t[2] + 50.0 EC1t[3] + 50.0 EC1t[4] + 50.0 EC1t[5] + 50.0 EC1t[6] + 50.0 EC1t[7] + 50.0 EC1t[8] + 100.0 EC2t[1] + 100.0 EC2t[2] + 100.0 EC2t[3] + 100.0 EC2t[4] + 100.0 EC2t[5] + 100.0 EC2t[6] + 100.0 EC2t[7] + 100.0 EC2t[8] + 400.0 C1t[1] + 400.0 C1t[2] + 400.0 C1t[3] + 400.0 C1t[4] + 400.0 C1t[5] + 400.0 C1t[6] + 400.0 C1t[7] + 400.0 C1t[8] + 350.0 C2t[1] + 350.0 C2t[2] + 350.0 C2t[3] + 350.0 C2t[4] + 350.0 C2t[5] + 350.0 C2t[6] + 350.0 C2t[7] + 350.0 C2t[8]>
Subject To
   R0 : <gurobi.LinExpr: -1.0 EP1t[1] + EP1t[2] + -1.0 P1t[2]> = 100.0
   R1 : <gurobi.LinExpr: -1.0 EP1t[2] + EP1t[3] + -1.0 P1t[3]> = 440.0
   R2 : <gurobi.LinExpr: -1.0 EP1t[3] + EP1t[4] + -1.0 P1t[4]> = 250.0
   R

In [8]:
print("Valor da função objetivo: "+str(round(model.ObjVal, 2)))
for v in model.getVars():
    print(str(v.varName)+" = "+str(round(v.x,2)))

Valor da função objetivo: 7444800.0
EP1t[1] = 0.0
EP1t[2] = 100.0
EP1t[3] = 540.0
EP1t[4] = 790.0
EP1t[5] = 1010.0
EP1t[6] = 1360.0
EP1t[7] = 1540.0
EP1t[8] = 1790.0
EP2t[1] = 0.0
EP2t[2] = 200.0
EP2t[3] = 1080.0
EP2t[4] = 1580.0
EP2t[5] = 2020.0
EP2t[6] = 2720.0
EP2t[7] = 3080.0
EP2t[8] = 3580.0
EC1t[1] = 800.0
EC1t[2] = 1100.0
EC1t[3] = 2420.0
EC1t[4] = 3170.0
EC1t[5] = 3830.0
EC1t[6] = 4880.0
EC1t[7] = 5420.0
EC1t[8] = 6170.0
EC2t[1] = -0.0
EC2t[2] = 0.0
EC2t[3] = 0.0
EC2t[4] = 0.0
EC2t[5] = 0.0
EC2t[6] = 0.0
EC2t[7] = 0.0
EC2t[8] = 0.0
P1t[1] = 800.0
P1t[2] = -0.0
P1t[3] = -0.0
P1t[4] = -0.0
P1t[5] = -0.0
P1t[6] = -0.0
P1t[7] = -0.0
P1t[8] = -0.0
P2t[1] = 400.0
P2t[2] = -0.0
P2t[3] = -0.0
P2t[4] = -0.0
P2t[5] = -0.0
P2t[6] = -0.0
P2t[7] = -0.0
P2t[8] = -0.0
C1t[1] = -0.0
C1t[2] = -0.0
C1t[3] = 0.0
C1t[4] = 0.0
C1t[5] = 0.0
C1t[6] = 0.0
C1t[7] = 0.0
C1t[8] = 0.0
C2t[1] = 800.0
C2t[2] = -0.0
C2t[3] = -0.0
C2t[4] = -0.0
C2t[5] = -0.0
C2t[6] = -0.0
C2t[7] = -0.0
C2t[8] = 0.0
CFt[1] = 1